In [37]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
import os
os.chdir('..')
sys.path.insert(0, os.getcwd())
import numpy as np
from Data_loader import load_subjects_from_json, get_all_npy_paths_by_group, base_folders

import math
import torch
from time import time
from torch.utils.data import Dataset, DataLoader


In [2]:
print(os.getcwd())

/home/dmartinez/Gait-Stability


In [3]:
from AE_pipeline_pytorch import (
    LSTMAutoencoder,
    BiLSTMAutoencoder,
    create_dataloader,
    create_test_dataloader,
    get_dataloaders,

    train_autoencoder,
    evaluate_and_detect,
    extract_and_save_latents,
    reconstruct_and_evaluate 
)

2025-06-27 08:57:16.826811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751025436.842088   19661 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751025436.846714   19661 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751025436.858247   19661 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751025436.858265   19661 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751025436.858267   19661 computation_placer.cc:177] computation placer alr

Using device: cuda


In [4]:
BATCH_SIZE = 256
NUM_BIOMECHANICAL_VARIABLES = 321
n_timesteps= 100 #cycle is normalized to 100 points 

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
#Load subjects lists 
train_subjects = {
    "G01": load_subjects_from_json("G01_train_subjects.json"),
    "G03": load_subjects_from_json("G03_train_subjects.json")
}
val_subjects = {
    "G01": load_subjects_from_json("G01_validation_subjects.json"),
    "G03": load_subjects_from_json("G03_validation_subjects.json")
}
test_subjects = {
    "G01": load_subjects_from_json("G01_test_subjects.json"),
    "G03": load_subjects_from_json("G03_test_subjects.json")
}

In [7]:
# Generate routes .npy
train_npy = get_all_npy_paths_by_group(train_subjects, base_folders)
val_npy   = get_all_npy_paths_by_group(val_subjects,   base_folders)
test_npy  = get_all_npy_paths_by_group(test_subjects,  base_folders)

print(f"Train .npy: {len(train_npy)} files")
print(f" Val  .npy: {len(val_npy)} files")
print(f" Test .npy: {len(test_npy)} files")

Train .npy: 932 files
 Val  .npy: 196 files
 Test .npy: 169 files


In [ ]:
#Just once Build Zarr datasets
import numpy as np
import zarr
import time
import logging
from tqdm import tqdm
import psutil  # pip install psutil if you want memory stats

def build_zarr_from_list(npy_paths, store_path, chunk_cycles=1000):
    # ——— SET UP LOGGING ———
    logging.basicConfig(
        format="%(asctime)s %(levelname)s: %(message)s",
        level=logging.INFO,
        datefmt="%H:%M:%S",
    )
    logging.info(f"Starting Zarr build → {store_path}")

    # ——— 1) COUNT TOTAL CYCLES ———
    counts = [np.load(p, mmap_mode="r").shape[0] for p in npy_paths]
    total = sum(counts)
    logging.info(f"Total cycles to write: {total}")

    # ——— 2) CREATE ZARR STORE ———
    store = zarr.DirectoryStore(store_path)
    root  = zarr.group(store=store, overwrite=True)
    arr   = root.zeros(
        "data",
        shape=(total, 100, 326),
        chunks=(chunk_cycles, 100, 326),
        dtype="float32"
    )

    # ——— 3) FILL IT ———
    offset = 0
    for path, cnt in tqdm(zip(npy_paths, counts),
                          total=len(npy_paths),
                          desc="Files",
                          unit="file"):
        t0 = time.perf_counter()
        data = np.load(path, mmap_mode="r").astype(np.float32)

        # optional: break large writes into sub‐chunks so you can see finer progress
        sub_chunk = max(cnt // 10, 1)
        for start in tqdm(range(0, cnt, sub_chunk),
                          desc="  cycles",
                          leave=False,
                          unit="cycle"):
            end = min(start + sub_chunk, cnt)
            arr[offset + start : offset + end] = data[start:end]

        offset += cnt
        dt = time.perf_counter() - t0

        # log memory usage (RSS) in MB
        mem = psutil.Process().memory_info().rss / 1024**2
        pct = offset / total * 100
        logging.info(
            f"Wrote {cnt} cycles from {path!r} in {dt:.1f}s "
            f"→ {offset}/{total} ({pct:.1f}%), RSS={mem:.0f} MB"
        )

    logging.info("✅ Zarr build complete")


In [ ]:
#Just once build Zarr datasets
build_zarr_from_list(train_npy, "train_cycles.zarr")
build_zarr_from_list(val_npy,   "val_cycles.zarr")
build_zarr_from_list(test_npy,  "test_cycles.zarr")

07:59:33 INFO: Starting Zarr build → test_cycles.zarr
07:59:52 INFO: Total cycles to write: 35115
Files: 100%|██████████| 169/169 [10:38<00:00,  3.78s/file]
08:10:31 INFO: ✅ Zarr build complete


In [6]:
#Data loader for Zarr datasets
import zarr
import torch
from torch.utils.data import Dataset

class GaitZarrDataset(Dataset):
    def __init__(self, store_path, return_meta=False):
        """
        store_path : path to one of your Zarr stores, e.g. "train_cycles.zarr"
        return_meta: if True, __getitem__ returns (features, meta)
        """
        self.return_meta = return_meta
        # only metadata and chunk-index loaded into RAM
        self._z = zarr.open(store_path, mode="r")["data"]
        self.total = self._z.shape[0]

    def __len__(self):
        return self.total

    def __getitem__(self, idx):
        # one C-level read of shape (100,326)
        data = self._z[idx]  # np.ndarray float32

        feat = torch.from_numpy(data[:, :321])    # (100,321)
        if self.return_meta:
            meta = torch.from_numpy(data[:, 321:]) # (100,5)
            return feat, meta
        else:
            return feat,feat


In [7]:
#Call data loader for Zarr datasets
from torch.utils.data import DataLoader

train_ds = GaitZarrDataset("train_cycles.zarr", return_meta=False)
train_loader = DataLoader(
    train_ds,
    batch_size=400,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=None,
)

val_ds = GaitZarrDataset("val_cycles.zarr", return_meta=False)
val_loader = DataLoader(
    val_ds,
    batch_size=4000,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
)

In [23]:
#check if the dataset is loaded correctly
for batch in train_loader:
    print(batch.shape)
    break

torch.Size([400, 100, 321])


In [8]:
 #=== Uso ===
train_loader, val_loader = get_dataloaders(train_npy, val_npy, 
                                        batch_size=100, num_workers=4, pin_memory=True) #, test_paths=test_npy)
#test_loader = get_dataloaders(train_paths, val_paths, test_paths=test_npy_paths)[2]#

In [24]:
# get me a sample of the dataloader
sample = next(iter(train_loader))
print(f"Sample shape: {sample.shape}")
# Preload the first batch to avoid delays in training

Sample shape: torch.Size([400, 100, 321])


In [9]:
# ─── Celda de profiling ─────────────────────────────────────────────────
import torch
import torch.nn.functional as F
import torch.profiler

def profile_training_step(model, train_loader, device, steps=5):
    activities = [
        torch.profiler.ProfilerActivity.CPU,
        torch.profiler.ProfilerActivity.CUDA,
    ]
    with torch.profiler.profile(
            activities=activities,
            record_shapes=True,
            profile_memory=True,
            with_stack=True
        ) as prof:
        for i, (x, _) in enumerate(train_loader):
            x = x.to(device)
            recon = model(x)
            loss  = F.mse_loss(recon, x)
            loss.backward()
            if i >= steps-1:
                break

    print(prof.key_averages().table(
        sort_by="cuda_time_total", row_limit=10
    ))

# Llamas al profiler antes de entrenar de verdad:
profile_training_step(model, train_loader, device, steps=5)


RuntimeError: DataLoader worker (pid(s) 238788, 238789, 238790, 238791, 238792, 238793, 238794, 238795) exited unexpectedly

In [ ]:
# en tu notebook


# crea el modelo y el train_loader como siempre…
model = LSTMAutoencoder(...).to(device)
train_loader = create_dataloader(...)

# 1) Perfila 5 batches
profile_training_step(model, train_loader, device, steps=5)

# 2) Una vez ajustes tu pipeline, ejecuta el entrenamiento normal:
train_autoencoder(model, train_loader, val_loader, run_id="exp1", epochs=20)

In [8]:
#Train 

model = LSTMAutoencoder(n_timesteps=100, n_vars=321, latent_dim=128).to(device)


# Compila el modelo (PyTorch 2.0 JIT), **antes** de entrenar
if hasattr(torch, "compile"):
    model = torch.compile(model)


# Tu función de train_autoencoder acepta train_loader y val_loader
train_autoencoder(
    model,
    train_loader,
    val_loader,
    run_id="Torch1",
    epochs=20,
    debug=True,
    debug_batches=60
)

[Epoch 1] Batch 0: I/O=56.646s, memcpy=0.012s, comp+back=0.337s
[Epoch 1] Batch 1: I/O=56.979s, memcpy=0.011s, comp+back=0.056s
[Epoch 1] Batch 2: I/O=59.016s, memcpy=0.007s, comp+back=0.054s


KeyboardInterrupt: 

In [ ]:
model.eval()
mse = torch.nn.MSELoss(reduction='none')

all_scores = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        recon = model(x)                       # (B, T, 321)
        # Si quieres comparar recon vs y, quizá tu función `evaluate_and_detect`
        # espera (x, y) y devuelve errores. Sino:
        # ejemplo de score: media de MSE por variable
        loss = mse(recon, x).mean(dim=(1,2))   # reconstrucción de x vs x
        # o si tu supervisión es recon vs y:
        # loss = mse(recon, y).mean(dim=(1,2))
        all_scores.append(loss.cpu())

all_scores = torch.cat(all_scores)
print("Test MSE (batch-wise):", all_scores.mean().item())
